# Limpieza de datos sobre avisos laborales, postulantes y postulaciones

In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn

## Lectura de datos

### Datos originales

In [47]:
# Postulantes
postulantes_educacion = pd.read_csv(\
    './data/fiuba_1_postulantes_educacion.csv', low_memory=False)
postulantes_sexo_y_edad = pd.read_csv(\
    './data/fiuba_2_postulantes_genero_y_edad.csv', low_memory=False)

# Avisos
avisos_online = pd.read_csv(\
    './data/fiuba_5_avisos_online.csv', low_memory=False)
avisos_detalle = pd.read_csv(\
    './data/fiuba_6_avisos_detalle.csv', low_memory=False)

# Transacciones
vistas = pd.read_csv(\
    './data/fiuba_3_vistas.csv', low_memory=False)
postulaciones = pd.read_csv(\
    './data/fiuba_4_postulaciones.csv', low_memory=False)

### Datos hasta el 15 de abril

In [48]:
# Postulantes
h15_postulantes_educacion = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_1_postulantes_educacion.csv',\
    low_memory=False)
h15_postulantes_sexo_y_edad = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_2_postulantes_genero_y_edad.csv',\
    low_memory=False)

# Avisos
h15_avisos_online = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_5_avisos_online.csv',\
    low_memory=False)
h15_avisos_detalle = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_6_avisos_detalle.csv',\
    low_memory=False)

# Transacciones
h15_vistas = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_3_vistas.csv',\
    low_memory=False)
h15_postulaciones = pd.read_csv(\
    './data/fiuba_hasta_15_abril/entrega6/fiuba_4_postulaciones.csv',\
    low_memory=False)

### Datos desde el 15 de abril

In [49]:
# Postulantes
d15_postulantes_educacion = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv',\
    low_memory=False)
d15_postulantes_sexo_y_edad = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv',\
    low_memory=False)

# Avisos
d15_avisos_detalle = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_6_avisos_detalle.csv',\
    low_memory=False)

# Transacciones
d15_vistas = pd.read_csv(\
    './data/fiuba_desde_15_Abril/fiuba_3_vistas.csv',\
    low_memory=False)

### Datos de avisos faltantes

In [50]:
# Avisos
m_avisos_detalle = pd.read_csv(\
    './data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv',\
    low_memory=False)

## Procesamiento de los datos de postulantes

Agregamos columna con ponderación del nivel de estudios alcanzado.

In [6]:
def categorizar_estudios(x):
    if((x.nombre is np.nan) | (x.nombre == 'Otro')):
        return 0
    if(x.nombre == 'Secundario'):
        nivel = 1
    if(x.nombre == 'Terciario/Técnico'):
        nivel = 2
    if(x.nombre == 'Universitario'):
        nivel = 3
    if((x.nombre == 'Posgrado') | (x.nombre == 'Master') |\
       (x.nombre == 'Doctorado')):
        nivel = 4
    if(x.estado == 'Graduado'):
        return nivel + 1
    if(x.estado == 'En Curso'):
        return nivel + 0.5
    return nivel

postulantes_educacion['nivel_estudios'] = postulantes_educacion\
    .apply(categorizar_estudios, axis=1)
h15_postulantes_educacion['nivel_estudios'] = h15_postulantes_educacion\
    .apply(categorizar_estudios, axis=1)
d15_postulantes_educacion['nivel_estudios'] = d15_postulantes_educacion\
    .apply(categorizar_estudios, axis=1)

Eliminamos las columnas ```nombre``` y ```estado``` dejando solamente la nueva columna   ```nivel_estudios``` con valores numéricos jerarquizados.

In [7]:
postulantes_educacion.drop(['nombre','estado'], axis=1, inplace=True)
h15_postulantes_educacion.drop(['nombre','estado'], axis=1, inplace=True)
d15_postulantes_educacion.drop(['nombre','estado'], axis=1, inplace=True)

Unimos todos los dataframes de la educación de los postulantes y nos quedamos solo con los registros del máximo nivel de estudios alcanzado.

In [8]:
m_postulantes_educacion = pd.concat([postulantes_educacion, \
                                 h15_postulantes_educacion, \
                                 d15_postulantes_educacion])

m_postulantes_educacion = m_postulantes_educacion \
    .sort_values('nivel_estudios', ascending = False) \
    .drop_duplicates('idpostulante')

m_postulantes_educacion.head(2)

,idpostulante,nivel_estudios
182965,61M19R,5.0
163713,Eze1oeN,5.0


In [10]:
m_postulantes_educacion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 447909 entries, 182965 to 179134
Data columns (total 2 columns):
idpostulante      447909 non-null object
nivel_estudios    447909 non-null float64
dtypes: float64(1), object(1)
memory usage: 10.3+ MB


Unión de los dataframes de sexo y edad de los postulantes

In [11]:
m_postulantes_sexo_y_edad = pd.concat([postulantes_sexo_y_edad, \
                                      h15_postulantes_sexo_y_edad, \
                                      d15_postulantes_sexo_y_edad])

Eliminamos las filas completamente iguales.

In [12]:
m_postulantes_sexo_y_edad = m_postulantes_sexo_y_edad.drop_duplicates()
m_postulantes_sexo_y_edad.count()

idpostulante       505382
fechanacimiento    478699
sexo               505382
dtype: int64

Pasamos el formato de la columna ```fechanacimiento``` a datetime convirtiendo los valores inválidos a ```NaN```.

In [13]:
m_postulantes_sexo_y_edad['fechanacimiento'] = \
    pd.to_datetime(postulantes_sexo_y_edad['fechanacimiento']\
                   , errors='coerce')
m_postulantes_sexo_y_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505382 entries, 0 to 281384
Data columns (total 3 columns):
idpostulante       505382 non-null object
fechanacimiento    384817 non-null datetime64[ns]
sexo               505382 non-null object
dtypes: datetime64[ns](1), object(2)
memory usage: 15.4+ MB


Agregamos una columna calculando la edad a partir de la fecha de nacimiento y la fecha actual. Si la edad resultante es menor a 16 o mayor a 100 no la tenemos en cuenta.

In [14]:
def calcular_edad(fecha_nacimiento):
    hoy = datetime.date.today()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day)\
            < (fecha_nacimiento.month, fecha_nacimiento.day))
    return np.NaN if ((edad > 100) | (edad < 16)) else edad

m_postulantes_sexo_y_edad['edad'] = \
    m_postulantes_sexo_y_edad['fechanacimiento']\
    .apply(lambda x: calcular_edad(x))

Llenamos las edades nulas con el valor de la edad promedio de las demás y borramos la columna ```fechanacimiento```.

In [15]:
edad_promedio = m_postulantes_sexo_y_edad['edad'].mean()
m_postulantes_sexo_y_edad['edad'].fillna(edad_promedio, inplace = True)
m_postulantes_sexo_y_edad.drop(['fechanacimiento'], axis=1, inplace=True)
m_postulantes_sexo_y_edad.head(2)

,idpostulante,sexo,edad
0,NM5M,FEM,47.0
1,5awk,FEM,55.0


Como existen postulantes repetidos con distintos sexos, nos quedamos arbitrariamente con alguno de ellos solamente.

In [16]:
m_postulantes_sexo_y_edad = \
    sklearn.utils.shuffle(m_postulantes_sexo_y_edad)
m_postulantes_sexo_y_edad.drop_duplicates('idpostulante', inplace=True)

One Hot Encoding para el sexo de los postulantes.

In [17]:
m_postulantes_sexo_y_edad['femenino'] = m_postulantes_sexo_y_edad['sexo']\
    .apply(lambda x: 1 if x == 'FEM' else 0)
m_postulantes_sexo_y_edad['masculino'] = m_postulantes_sexo_y_edad['sexo']\
    .apply(lambda x: 1 if x == 'MASC' else 0)
m_postulantes_sexo_y_edad['no_declara_sexo'] = m_postulantes_sexo_y_edad['sexo']\
    .apply(lambda x: 1 if x == 'NO_DECLARA' else 0)

# Eliminamos la columna sexo
m_postulantes_sexo_y_edad.drop(['sexo'], axis=1, inplace=True)
m_postulantes_sexo_y_edad.head(2)

,idpostulante,edad,femenino,masculino,no_declara_sexo
255044,akD8QZ0,29.966324,0,1,0
158448,bOjGl2b,22.000000,0,1,0


In [18]:
m_postulantes_sexo_y_edad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 255044 to 124854
Data columns (total 5 columns):
idpostulante       504407 non-null object
edad               504407 non-null float64
femenino           504407 non-null int64
masculino          504407 non-null int64
no_declara_sexo    504407 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 23.1+ MB


Unimos ambos dataframes de los postulantes.

In [19]:
m_postulantes = pd.merge(m_postulantes_sexo_y_edad, m_postulantes_educacion, \
                       on = 'idpostulante', how='outer')
m_postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 6 columns):
idpostulante       504407 non-null object
edad               504407 non-null float64
femenino           504407 non-null int64
masculino          504407 non-null int64
no_declara_sexo    504407 non-null int64
nivel_estudios     447909 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 26.9+ MB


Como existen postulantes sin nivel de estudios declarados, completamos con la mediana.

In [20]:
mediana_estudios = m_postulantes['nivel_estudios'].median()
print('Mediana del nivel de estudios: ', mediana_estudios)

Mediana del nivel de estudios:  3.0


In [21]:
m_postulantes['nivel_estudios'].fillna(mediana_estudios, inplace=True)
m_postulantes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 6 columns):
idpostulante       504407 non-null object
edad               504407 non-null float64
femenino           504407 non-null int64
masculino          504407 non-null int64
no_declara_sexo    504407 non-null int64
nivel_estudios     504407 non-null float64
dtypes: float64(2), int64(3), object(1)
memory usage: 26.9+ MB


In [22]:
m_postulantes.to_csv('./postulantes_encoding.csv', encoding='utf-8')

## Procesamiento de los datos de los avisos

In [51]:
avisos_detalle.head(2)

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste


Descartamos las columnas ```idpais```, porque es 1 para todos los avisos (Argentina); ```descripcion``` (por ahora, pero habría que procesarlo luego), ```nombre_zona``` ya que casi todos son de Buenos Aires y alrededores y ```ciudad``` y ```mapacalle``` que no agregan información relevante.

In [52]:
avisos_detalle.drop(['idpais','descripcion','nombre_zona',\
                     'ciudad','mapacalle'], axis=1, inplace=True)
h15_avisos_detalle.drop(['idpais','descripcion','nombre_zona',\
                     'ciudad','mapacalle'], axis=1, inplace=True)
d15_avisos_detalle.drop(['idpais','descripcion','nombre_zona',\
                     'ciudad','mapacalle'], axis=1, inplace=True)
m_avisos_detalle.drop(['idpais','descripcion','nombre_zona',\
                     'ciudad','mapacalle'], axis=1, inplace=True)

Unimos todos los avisos y eliminamos los duplicados.

In [53]:
m_avisos = pd.concat([avisos_detalle, h15_avisos_detalle, \
                      d15_avisos_detalle, m_avisos_detalle])

m_avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45969 entries, 0 to 337
Data columns (total 6 columns):
idaviso                 45969 non-null int64
titulo                  45969 non-null object
tipo_de_trabajo         45969 non-null object
nivel_laboral           45634 non-null object
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: int64(1), object(5)
memory usage: 2.5+ MB


In [54]:
m_avisos = m_avisos.drop_duplicates()
m_avisos = m_avisos.drop_duplicates('idaviso')
m_avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 0 to 337
Data columns (total 6 columns):
idaviso                 25288 non-null int64
titulo                  25288 non-null object
tipo_de_trabajo         25288 non-null object
nivel_laboral           24953 non-null object
nombre_area             25288 non-null object
denominacion_empresa    25281 non-null object
dtypes: int64(1), object(5)
memory usage: 1.4+ MB


Codificamos la columna de nivel laboral.

In [55]:
m_avisos.nivel_laboral.value_counts()

Senior / Semi-Senior                    16983
Junior                                   4149
Otro                                     1972
Jefe / Supervisor / Responsable          1527
Gerencia / Alta Gerencia / Dirección      322
Name: nivel_laboral, dtype: int64

Como la mayoría son "Senior / Semi-Senior", le asignaremos ese valor a los que tienen el valor "Otro".

In [56]:
def categorizar_nivel_laboral(x):
    if(x == 'Junior'):
        return 1
    if(x == 'Senior / Semi-Senior'):
        return 2
    if(x == 'Jefe / Supervisor / Responsable'):
        return 3
    if(x == 'Gerencia / Alta Gerencia / Dirección'):
        return 4
    return 0

m_avisos['nivel_laboral'] = m_avisos['nivel_laboral']\
    .apply(lambda x: categorizar_nivel_laboral(x))

In [57]:
m_avisos.tipo_de_trabajo.value_counts()

Full-time          22830
Part-time           1746
Teletrabajo          248
Por Horas            127
Pasantia             119
Temporario            95
Por Contrato          88
Fines de Semana       28
Primer empleo          6
Voluntario             1
Name: tipo_de_trabajo, dtype: int64

In [58]:
def categorizar_tipo_de_trabajo(x):
    if((x == 'Primer empleo') | (x == 'Pasantia') | (x == 'Voluntario')):
        return 0
    if((x == 'Teletrabajo') | (x == 'Por Horas') | (x == 'Temporario')\
      | (x == 'Por Contrato') | (x == 'Fines de Semana')):
        return 1
    if(x == 'Part-time'):
        return 2
    if(x == 'Full-time'):
        return 3

m_avisos['tipo_de_trabajo'] = m_avisos['tipo_de_trabajo']\
    .apply(lambda x: categorizar_tipo_de_trabajo(x))

In [59]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')

In [60]:
m_avisos['titulo'] = m_avisos.titulo\
    .apply(lambda x: stemmer.stem(x.split()[0]))

In [61]:
del m_avisos['denominacion_empresa']
m_avisos.head(2)

,idaviso,titulo,tipo_de_trabajo,nivel_laboral,nombre_area
0,8725750,vendedor/,3,2,Comercial
1,17903700,enfermer,3,2,Salud


In [62]:
m_avisos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 0 to 337
Data columns (total 5 columns):
idaviso            25288 non-null int64
titulo             25288 non-null object
tipo_de_trabajo    25288 non-null int64
nivel_laboral      25288 non-null int64
nombre_area        25288 non-null object
dtypes: int64(3), object(2)
memory usage: 1.2+ MB


Unimos los dos archivos de avisos online y se lo agregamos como columna binaria al dataframe de avisos.

In [33]:
'''
avisos_online = pd.concat([avisos_online, \
                           h15_avisos_online])

avisos_online.drop_duplicates()

avisos_online['online'] = 1
avisos_online.info()
'''

"\navisos_online = pd.concat([avisos_online,                            h15_avisos_online])\n\navisos_online.drop_duplicates()\n\navisos_online['online'] = 1\navisos_online.info()\n"

In [94]:
'''
m_avisos = pd.merge(m_avisos, avisos_online, \
                    on = 'idaviso', how = 'left')
# Agrego 0 para los avisos que no están online
m_avisos['online'].fillna(0, inplace = True)
'''

Label encoding para los títulos, nombres de área y de empresa.

In [64]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [65]:
avisos_encoding = MultiColumnLabelEncoder(columns = \
    ['titulo','nombre_area'])\
    .fit_transform(m_avisos)
avisos_encoding.head(2)

,idaviso,titulo,tipo_de_trabajo,nivel_laboral,nombre_area
0,8725750,1551,3,2,30
1,17903700,608,3,2,158


In [66]:
avisos_encoding.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 0 to 337
Data columns (total 5 columns):
idaviso            25288 non-null int64
titulo             25288 non-null int64
tipo_de_trabajo    25288 non-null int64
nivel_laboral      25288 non-null int64
nombre_area        25288 non-null int64
dtypes: int64(5)
memory usage: 1.2 MB


In [67]:
avisos_encoding.to_csv('./avisos_encoding.csv', encoding='utf-8')

## Unión de las postulaciones

In [68]:
m_postulaciones = pd.concat([postulaciones, h15_postulaciones])

In [69]:
m_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8311264 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    object
dtypes: int64(1), object(2)
memory usage: 253.6+ MB


In [70]:
m_postulaciones = m_postulaciones.drop_duplicates()
m_postulaciones.count()

idaviso             6604534
idpostulante        6604534
fechapostulacion    6604534
dtype: int64

In [71]:
m_postulaciones['fechapostulacion'] = \
    pd.to_datetime(m_postulaciones['fechapostulacion'], errors='coerce')
m_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6604534 entries, 0 to 4909640
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 201.6+ MB


In [72]:
m_postulaciones.count()

idaviso             6604534
idpostulante        6604534
fechapostulacion    6604534
dtype: int64

In [73]:
m_postulaciones = pd.merge(m_postulaciones, m_postulantes, \
                  on = 'idpostulante', how = 'inner')
m_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6604534 entries, 0 to 6604533
Data columns (total 8 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
edad                float64
femenino            int64
masculino           int64
no_declara_sexo     int64
nivel_estudios      float64
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 453.5+ MB


In [74]:
m_postulaciones = pd.merge(m_postulaciones, avisos_encoding, on = 'idaviso', how = 'inner')
m_postulaciones.head(2)

,idaviso,idpostulante,fechapostulacion,edad,femenino,masculino,no_declara_sexo,nivel_estudios,titulo,tipo_de_trabajo,nivel_laboral,nombre_area
0,1112257047,NM5M,2018-01-15 16:22:34,47.0,1,0,0,2.0,1312,2,0,14
1,1112257047,1kJqGb,2018-01-15 10:23:11,34.0,1,0,0,3.5,1312,2,0,14


In [75]:
m_postulaciones.count()

idaviso             6188634
idpostulante        6188634
fechapostulacion    6188634
edad                6188634
femenino            6188634
masculino           6188634
no_declara_sexo     6188634
nivel_estudios      6188634
titulo              6188634
tipo_de_trabajo     6188634
nivel_laboral       6188634
nombre_area         6188634
dtype: int64

In [76]:
m_postulaciones.to_csv('./postulaciones_encoding.csv', encoding='utf-8')

## Postulaciones y no postulaciones

In [77]:
avisos_encoding.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25288 entries, 0 to 337
Data columns (total 5 columns):
idaviso            25288 non-null int64
titulo             25288 non-null int64
tipo_de_trabajo    25288 non-null int64
nivel_laboral      25288 non-null int64
nombre_area        25288 non-null int64
dtypes: int64(5)
memory usage: 1.2 MB


In [78]:
avisos_g = avisos_encoding
while len(avisos_g) < 504407:
    avisos_g = pd.concat([avisos_g, avisos_encoding])
avisos_g.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505760 entries, 0 to 337
Data columns (total 5 columns):
idaviso            505760 non-null int64
titulo             505760 non-null int64
tipo_de_trabajo    505760 non-null int64
nivel_laboral      505760 non-null int64
nombre_area        505760 non-null int64
dtypes: int64(5)
memory usage: 23.2 MB


In [79]:
avisos_g = avisos_g[:504407]
avisos_g = sklearn.utils.shuffle(avisos_g)
avisos_g.count()

idaviso            504407
titulo             504407
tipo_de_trabajo    504407
nivel_laboral      504407
nombre_area        504407
dtype: int64

In [80]:
avisos_g = avisos_g.reset_index().drop('index',axis=1)

In [82]:
no_postulaciones = pd.concat([m_postulantes, avisos_g], axis = 1)
no_postulaciones.head(2)

,idpostulante,edad,femenino,masculino,no_declara_sexo,nivel_estudios,idaviso,titulo,tipo_de_trabajo,nivel_laboral,nombre_area
0,akD8QZ0,29.966324,0,1,0,3.0,1111946901,1548,3,0,185
1,bOjGl2b,22.000000,0,1,0,2.0,1111199358,787,3,2,90


In [83]:
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 11 columns):
idpostulante       504407 non-null object
edad               504407 non-null float64
femenino           504407 non-null int64
masculino          504407 non-null int64
no_declara_sexo    504407 non-null int64
nivel_estudios     504407 non-null float64
idaviso            504407 non-null int64
titulo             504407 non-null int64
tipo_de_trabajo    504407 non-null int64
nivel_laboral      504407 non-null int64
nombre_area        504407 non-null int64
dtypes: float64(2), int64(8), object(1)
memory usage: 46.2+ MB


In [84]:
no_postulaciones = no_postulaciones.drop_duplicates()
no_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 504407 entries, 0 to 504406
Data columns (total 11 columns):
idpostulante       504407 non-null object
edad               504407 non-null float64
femenino           504407 non-null int64
masculino          504407 non-null int64
no_declara_sexo    504407 non-null int64
nivel_estudios     504407 non-null float64
idaviso            504407 non-null int64
titulo             504407 non-null int64
tipo_de_trabajo    504407 non-null int64
nivel_laboral      504407 non-null int64
nombre_area        504407 non-null int64
dtypes: float64(2), int64(8), object(1)
memory usage: 46.2+ MB


In [85]:
no_postulaciones['sepostulo'] = 0
m_postulaciones['sepostulo'] = 1

In [86]:
m_postulaciones = m_postulaciones.drop_duplicates(['idpostulante','idaviso'])
m_postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6187929 entries, 0 to 6188633
Data columns (total 13 columns):
idaviso             int64
idpostulante        object
fechapostulacion    datetime64[ns]
edad                float64
femenino            int64
masculino           int64
no_declara_sexo     int64
nivel_estudios      float64
titulo              int64
tipo_de_trabajo     int64
nivel_laboral       int64
nombre_area         int64
sepostulo           int64
dtypes: datetime64[ns](1), float64(2), int64(9), object(1)
memory usage: 660.9+ MB


In [87]:
del m_postulaciones['fechapostulacion']
m_postulaciones.head(2)

,idaviso,idpostulante,edad,femenino,masculino,no_declara_sexo,nivel_estudios,titulo,tipo_de_trabajo,nivel_laboral,nombre_area,sepostulo
0,1112257047,NM5M,47.0,1,0,0,2.0,1312,2,0,14,1
1,1112257047,1kJqGb,34.0,1,0,0,3.5,1312,2,0,14,1


In [88]:
entrenamiento = pd.concat([no_postulaciones, m_postulaciones])
entrenamiento.count()

edad               6692336
femenino           6692336
idaviso            6692336
idpostulante       6692336
masculino          6692336
nivel_estudios     6692336
nivel_laboral      6692336
no_declara_sexo    6692336
nombre_area        6692336
sepostulo          6692336
tipo_de_trabajo    6692336
titulo             6692336
dtype: int64

In [89]:
entrenamiento.head(2)

,edad,femenino,idaviso,idpostulante,masculino,nivel_estudios,nivel_laboral,no_declara_sexo,nombre_area,sepostulo,tipo_de_trabajo,titulo
0,29.966324,0,1111946901,akD8QZ0,1,3.0,0,0,185,0,3,1548
1,22.000000,0,1111199358,bOjGl2b,1,2.0,2,0,90,0,3,787


In [90]:
train = entrenamiento.sort_values('sepostulo', ascending=False)\
    .drop_duplicates(['idpostulante','idaviso'])

train.groupby(['idpostulante','idaviso'])\
    .count().sort_values(by='edad',ascending=False).head()

edad  femenino  masculino  nivel_estudios  \
idpostulante idaviso                                                 
0z5Dmrd      1111815087     1         1          1               1   
aE4GbW       1112264349     1         1          1               1   
             1112266777     1         1          1               1   
             1112266524     1         1          1               1   
             1112265973     1         1          1               1   

                         nivel_laboral  no_declara_sexo  nombre_area  \
idpostulante idaviso                                                   
0z5Dmrd      1111815087              1                1            1   
aE4GbW       1112264349              1                1            1   
             1112266777              1                1            1   
             1112266524              1                1            1   
             1112265973              1                1            1   

                         sepostulo  tipo_de_trabajo  titulo  
idpostulante idaviso                                         
0z5Dmrd      1111815087          1                1       1  
aE4GbW       1112264349          1                1       1  
             1112266777          1                1       1  
             1112266524          1                1       1  
             1112265973          1                1       1

In [91]:
train.loc[(train.idpostulante == '0z5Dmrd'),:]

,edad,femenino,idaviso,idpostulante,masculino,nivel_estudios,nivel_laboral,no_declara_sexo,nombre_area,sepostulo,tipo_de_trabajo,titulo
2233446,24.0,0,1112384041,0z5Dmrd,1,4.0,2,0,154,1,3,714
1639945,24.0,0,1112420060,0z5Dmrd,1,4.0,2,0,154,1,3,480
391386,24.0,0,1111815087,0z5Dmrd,1,4.0,2,0,181,0,3,394


In [92]:
train.to_csv('./entrenamiento_05.csv', encoding='utf-8')

In [87]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12758097 entries, 6188633 to 6382083
Data columns (total 14 columns):
denominacion_empresa    int64
edad                    float64
estado_estudios         float64
femenino                int64
idaviso                 int64
idpostulante            object
masculino               int64
nivel_estudios          float64
nivel_laboral           int64
no_declara_sexo         int64
nombre_area             int64
sepostulo               int64
tipo_de_trabajo         int64
titulo                  int64
dtypes: float64(3), int64(10), object(1)
memory usage: 1.4+ GB
